<a href="https://colab.research.google.com/github/eman-kom/tensorflow-fundamentals/blob/main/08_NLP_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-001c0447-99b1-66bc-eab5-06e763b376c2)


In [51]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-07-03 06:41:44--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-07-03 06:41:44 (99.8 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [52]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [53]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

unzip_data("nlp_getting_started.zip")

--2022-07-03 06:41:44--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.115.128, 172.253.122.128, 172.253.63.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.115.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-07-03 06:41:44 (131 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



In [54]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [55]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [56]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [57]:
len(train_df), len(test_df)

(7613, 3263)

In [58]:
import random
random_index = random.randint(0, len(train_df)-5)

for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "<real disaster>" if target > 0 else "<not a real disaster>")
  print(f"Text\n{text}\n")
  print("---\n")

Target: 1 <real disaster>
Text
[infowars]  Nashville Theater Attack: Will Gun Grabbers Now Demand ÛÏHatchet Control?Û http://t.co/n3yJb8TcPm #nwo

---

Target: 1 <real disaster>
Text
Cross-border terrorism: Pakistan caught red-handed again http://t.co/uDj50J3MV4

---

Target: 1 <real disaster>
Text
Damn...was wondering where my drone ended up after the freak windstorm...?? https://t.co/dHgGxo7Mcc

---

Target: 0 <not a real disaster>
Text
i just drove with both my parents in the car lmao that was a panic attack waiting to happen

---

Target: 0 <not a real disaster>
Text
Oh and fuck Bill Clinton for bombing us and fuck NATO.

---



In [59]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    train_df_shuffled["text"].to_numpy(),
    train_df_shuffled["target"].to_numpy(),
    test_size=0.1,
    random_state=42
)

In [60]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

In [61]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [62]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

text_vectorizer = TextVectorization(
    max_tokens=10000,
    output_mode="int",
    output_sequence_length=15,
)

In [63]:
text_vectorizer.adapt(train_sentences)

In [64]:
text_vectorizer(["There's a flood in my street!"])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [65]:
random_sent = random.choice(train_sentences)
print(f"Before: {random_sent}")
text_vectorizer([random_sent])

Before: Cuban leader extends sympathy to Vietnam over flooding at http://t.co/QcyXwr2rdv


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[3956, 1648, 3867, 3360,    5, 1743,   60,  231,   17,    1,    0,
           0,    0,    0,    0]])>

In [66]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
len(words_in_vocab), top_5_words, bottom_5_words

(10000,
 ['', '[UNK]', 'the', 'a', 'in'],
 ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1'])

In [67]:
from tensorflow.keras import layers

embedding = layers.Embedding(
    input_dim=10000,
    output_dim=128,
    input_length=15
)

embedding

In [68]:
sample_embed = embedding(text_vectorizer([random.choice(train_sentences)]))
sample_embed

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.04206889,  0.04232413,  0.03249537, ..., -0.00448012,
         -0.0168712 ,  0.03429223],
        [ 0.0461624 , -0.04788089,  0.01327837, ...,  0.04680759,
         -0.0349403 ,  0.04377338],
        [ 0.02406976,  0.02649131,  0.01166637, ..., -0.02230825,
          0.02356965, -0.03319588],
        ...,
        [-0.02151077,  0.02080262, -0.01359285, ..., -0.0292204 ,
         -0.04757923, -0.00423776],
        [-0.02151077,  0.02080262, -0.01359285, ..., -0.0292204 ,
         -0.04757923, -0.00423776],
        [-0.02151077,  0.02080262, -0.01359285, ..., -0.0292204 ,
         -0.04757923, -0.00423776]]], dtype=float32)>

Model 0: Baseline Model -> Sklearn Multinomial Naive Bayes using TF-IDF

In [69]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [70]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Baseline Score: {baseline_score*100:.2f}%")

Baseline Score: 79.27%


In [71]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {
      "accuracy": model_accuracy,
      "precision": model_precision,
      "recall": model_recall,
      "f1": model_f1
  }

  return model_results

In [72]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:5]

array([1, 1, 1, 0, 0])

In [73]:
baseline_results = calculate_results(val_labels, baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [74]:
from helper_functions import create_tensorboard_callback

SAVE_DIR = "model_logs"

Model 1: Simple Dense Model

In [75]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_2   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [76]:
model_1.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [77]:
model_1_history = model_1.fit(
    x=train_sentences,
    y=train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, experiment_name="model_1_dense")]
)

Saving TensorBoard log files to: model_logs/model_1_dense/20220703-064146
Epoch 1/5
215/215 [==============================] - 2s 5ms/step - loss: 0.6102 - accuracy: 0.6971 - val_loss: 0.5335 - val_accuracy: 0.7690
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.4404 - accuracy: 0.8186 - val_loss: 0.4703 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.3462 - accuracy: 0.8597 - val_loss: 0.4623 - val_accuracy: 0.7887
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2842 - accuracy: 0.8905 - val_loss: 0.4631 - val_accuracy: 0.7861
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2374 - accuracy: 0.9104 - val_loss: 0.4759 - val_accuracy: 0.7795


In [78]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4759 - accuracy: 0.7795


[0.47593948245048523, 0.7795275449752808]

In [79]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs

array([[4.22900140e-01],
       [7.65965521e-01],
       [9.97654676e-01],
       [1.44564897e-01],
       [1.19687535e-01],
       [9.38139379e-01],
       [9.20077443e-01],
       [9.94916558e-01],
       [9.73630190e-01],
       [3.03095430e-01],
       [1.30886123e-01],
       [7.25487173e-01],
       [5.40875979e-02],
       [2.10359275e-01],
       [4.68070479e-03],
       [1.71248615e-01],
       [2.35347468e-02],
       [9.39325541e-02],
       [2.89501548e-01],
       [5.78399897e-01],
       [9.07607734e-01],
       [4.76164632e-02],
       [4.70919609e-01],
       [1.03937730e-01],
       [9.67124999e-01],
       [9.98988807e-01],
       [4.11257558e-02],
       [8.75935405e-02],
       [3.45983952e-02],
       [2.03646094e-01],
       [6.02001607e-01],
       [2.84807205e-01],
       [4.91012007e-01],
       [2.56407708e-01],
       [5.17343998e-01],
       [7.20154196e-02],
       [9.95332539e-01],
       [1.87184528e-01],
       [4.21825610e-02],
       [9.98569131e-01],


In [80]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [81]:
model_1_results = calculate_results(val_labels, model_1_preds)
model_1_results, baseline_results

({'accuracy': 77.95275590551181,
  'f1': 0.7774022539420016,
  'precision': 0.7816545659065345,
  'recall': 0.7795275590551181},
 {'accuracy': 79.26509186351706,
  'f1': 0.7862189758049549,
  'precision': 0.8111390004213173,
  'recall': 0.7926509186351706})

In [83]:
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
embed_weights.shape

(10000, 128)